In [21]:
import pandas as pd 
import numpy as np

visa_experience = '/Users/churnika/Desktop/Projects/ML-Crate/F1 Visa Experiences/Dataset/telegram.csv'
visa_data = pd.read_csv(visa_experience) 

visa_data_clean = visa_data.dropna() #removing rows with missing values

visa_data_clean.describe(include='all')

,msg_id,date,message
count,6391.000000,6391,6391
unique,NaN,6372,6317
top,NaN,2021-07-01 04:30:10,Look this out for visa alerts👇\nhttps://t.me/A...
freq,NaN,2,5
mean,3384.868252,NaN,NaN
std,1908.469258,NaN,NaN
min,4.000000,NaN,NaN
25%,1768.500000,NaN,NaN
50%,3401.000000,NaN,NaN
75%,5034.500000,NaN,NaN


In [22]:
# accessing the message column and converting it into lower case

msg_exp = visa_data_clean['message'].str.lower()

In [23]:
# removing the urls that are present.

import re

text_without_urls = []

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_without_urls = url_pattern.sub('',text)
    return text_without_urls

for i in range(len(msg_exp)):
    try:
        msg_exp[i] = remove_urls(msg_exp[i])
        text_without_urls.append(msg_exp[i])
    except KeyError as e:
        continue


In [24]:
# removing punctuations

import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

print("Length of text_without_urls:", len(text_without_urls))

for i in range(len(text_without_urls)):
    try:
        text_without_urls[i] = remove_punctuation(text_without_urls[i])

    except KeyError as e:
        print(f"Error at index {i}: {e}. Skipping...")
        continue



Length of text_without_urls: 6340


In [25]:
# tokenisation

import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Tokenize text and remove URLs
cleaned_tokens = []

for i in range(len(text_without_urls)):
    try:
        doc = nlp(text_without_urls[i])
        # Extract tokens from the document and filter out URLs
        tokens = [token.text for token in doc if not token.like_url]
        cleaned_tokens.append(tokens)
    except Exception as e:
        print(f"Error at index {i}: {e}. Skipping...")
        continue

# cleaned_tokens now contains the tokens without URLs for each text


In [27]:
import re

def remove_numbers(text):
    number_pattern = re.compile(r'\d+')
    tokens_without_numbers = number_pattern.sub('', text)
    return tokens_without_numbers

# Assuming cleaned_tokens is defined elsewhere in your code
for i in range(len(cleaned_tokens)):
    try:
        cleaned_tokens[i] = remove_numbers(cleaned_tokens[i])
    except Exception as e:
        print(f"Error at index {i}: {e}. Skipping...")
        continue


Error at index 0: expected string or bytes-like object. Skipping...
Error at index 1: expected string or bytes-like object. Skipping...
Error at index 2: expected string or bytes-like object. Skipping...
Error at index 3: expected string or bytes-like object. Skipping...
Error at index 4: expected string or bytes-like object. Skipping...
Error at index 5: expected string or bytes-like object. Skipping...
Error at index 6: expected string or bytes-like object. Skipping...
Error at index 7: expected string or bytes-like object. Skipping...
Error at index 8: expected string or bytes-like object. Skipping...
Error at index 9: expected string or bytes-like object. Skipping...
Error at index 10: expected string or bytes-like object. Skipping...
Error at index 11: expected string or bytes-like object. Skipping...
Error at index 12: expected string or bytes-like object. Skipping...
Error at index 13: expected string or bytes-like object. Skipping...
Error at index 14: expected string or bytes-